# Creating a cluster/slice

## Please read carefully all the instructions. Specially the comments given in each cell. These comments are pertinent to run a cluster successfully. 

If the fabric environment is not set please read through the topic <code>FABRIC Environment Setup</code> in <code>start_here.ipynb</code> notebook given under <code>jupyter-example</code> folder in fabric's JupyterHub.

<b> At first read the comments in the cell and then only run the cell. </b>

## STEP-1

In [1]:
# Run this cell
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
try:
    fablib = fablib_manager()
    fablib.show_config()
except Exception as e:
    print(f"Exception: {e}")

User: ajay.kumar@missouri.edu bastion key is valid!
Configuration is valid


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,68926660-da26-475d-9c40-50ebf0a5a812
Bastion Host,bastion.fabric-testbed.net
Bastion Username,ajaykumar_0000102567
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


In [158]:
# slice_name='GPU_Variant_Calling_FIU'
# node_name='Node2'
# slice = fablib.get_slice(slice_name)
# node = slice.get_node(node_name)
# node.os_reboot()

In [2]:
# Run this cell
# To find all the available resources at this time
# run the cell below for easier readability 
try:
    resources = fablib.list_sites()
except Exception as e:
    print(f"Exception: {e}")

Name,State,Address,Location,PTP Capable,Hosts,CPUs,Cores Available,Cores Capacity,Cores Allocated,Ram Available,Ram Capacity,Ram Allocated,Disk Available,Disk Capacity,Disk Allocated,Basic NIC Available,Basic NIC Capacity,Basic NIC Allocated,P4-Switch Available,P4-Switch Capacity,P4-Switch Allocated,ConnectX-6 Available,ConnectX-6 Capacity,ConnectX-6 Allocated,ConnectX-5 Available,ConnectX-5 Capacity,ConnectX-5 Allocated,ConnectX-7-100 Available,ConnectX-7-100 Capacity,ConnectX-7-100 Allocated,ConnectX-7-400 Available,ConnectX-7-400 Capacity,ConnectX-7-400 Allocated,BlueField2-ConnectX-6 Available,BlueField2-ConnectX-6 Capacity,BlueField2-ConnectX-6 Allocated,NVMe Available,NVMe Capacity,NVMe Allocated,Tesla T4 Available,Tesla T4 Capacity,Tesla T4 Allocated,RTX6000 Available,RTX6000 Capacity,RTX6000 Allocated,A30 Available,A30 Capacity,A30 Allocated,A40 Available,A40 Capacity,A40 Allocated,U280 Available,U280 Capacity,U280 Allocated,SN1022 Available,SN1022 Capacity,SN1022 Allocated
LOSA,Active,"818 West 7th Street,Los Angeles, CA 90017","(34.0490803, -118.259534)",False,3,6,248,384,136,1130,1434,304,53798,56618,2820,357,381,24,0,0,0,1,2,1,2,2,0,1,1,0,0,0,0,0,0,0,0,10,10,0,0,0,0,0,0,4,4,0,0,0,0,1,1,0,0,0,0
EDUKY,Active,"301 Hilltop Avenue,Lexington, KY 40506","(38.0325, -84.502801)",True,18,36,73700,73728,28,8508,8604,96,133952,134082,130,4436,4445,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UCSD,Active,"10100 Hopkins Drive,CA 92093","(32.8886802, -117.239324)",True,5,10,387,640,253,1830,2390,560,54707,58717,4010,540,635,95,1,1,0,1,2,1,4,4,0,0,0,0,0,0,0,0,0,0,3,16,13,4,4,0,3,6,3,0,0,0,0,0,0,1,1,0,0,0,0
STAR,Active,"710 North Lakeshore Drive, 60611","(42.2359989, -88.1575427)",True,6,12,369,768,399,1528,2820,1292,100126,109696,9570,643,762,119,0,1,1,1,2,1,4,6,2,0,0,0,0,0,0,0,0,0,8,20,12,0,6,6,2,6,4,0,0,0,0,0,0,1,1,0,0,0,0
CERN,Active,"Meyrin site, Auvergne-Rhone-Alpes, Metropolitan France, 01280, France","(46.2338702, 6.0469869)",True,6,12,460,768,308,2148,2868,720,46883,54003,7120,722,762,40,0,0,0,1,6,5,4,4,0,0,0,0,0,0,0,0,0,0,7,22,15,0,0,0,0,0,0,3,5,2,0,2,2,0,0,0,0,0,0
PSC,Active,"4350 Northern Pike,Monroeville, PA 15146","(40.4343887, -79.750207)",True,3,6,381,384,3,1422,1434,12,56598,56618,20,380,381,1,0,0,0,2,2,0,2,2,0,0,0,0,0,0,0,0,0,0,10,10,0,0,0,0,0,0,0,4,4,0,0,0,0,1,1,0,0,0,0
TACC,Active,"10100 Burnet Rd,Austin, TX 78758","(30.3899405, -97.7261807)",False,5,10,189,640,451,1782,2390,608,104653,107463,2810,436,635,199,0,0,0,2,2,0,4,4,0,0,0,0,0,0,0,0,0,0,16,16,0,4,4,0,6,6,0,0,0,0,0,0,0,0,1,1,0,0,0
TOKY,Active,"7-3-1 Hongo,Bunkyo-ku,Tokyo,Japan","(35.7115097, 139.7640619)",True,3,6,356,384,28,1370,1434,64,62898,64068,1170,379,381,2,0,0,0,3,3,0,2,2,0,0,0,0,0,0,0,0,0,0,10,10,0,0,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0
HAWI,Active,"2520 Correa Road,Honolulu, Hi 96822","(21.2989762, -157.8163991)",True,5,10,504,640,136,1846,2390,544,106593,107463,870,626,635,9,0,0,0,2,2,0,4,4,0,1,1,0,0,0,0,0,0,0,16,16,0,0,0,0,0,0,0,0,8,8,0,0,0,1,1,0,0,0,0
MASS,Active,"100 Bigelow Street,Holyoke, MA 01040","(42.202493, -72.6078766)",True,3,6,366,384,18,1394,1434,40,63058,64068,1010,378,381,3,0,0,0,2,2,0,2,2,0,1,1,0,0,0,0,0,0,0,10,10,0,2,2,0,3,3,0,0,0,0,0,0,0,1,1,0,0,0,0


In [3]:
# for easier readability of available resources
import pandas as pd 
pd.DataFrame(resources.data)[['Hosts','CPUs','Name','Cores Available','Ram Available','Tesla T4 Available','RTX6000 Available','A30 Available','A40 Available']]


,Hosts,CPUs,Name,Cores Available,Ram Available,Tesla T4 Available,RTX6000 Available,A30 Available,A40 Available
0,3,6,LOSA,248,1130,0,0,4,0
1,18,36,EDUKY,73700,8508,0,0,0,0
2,5,10,UCSD,387,1830,4,3,0,0
3,6,12,STAR,369,1528,0,2,0,0
4,6,12,CERN,460,2148,0,0,3,0
5,3,6,PSC,381,1422,0,0,4,0
6,5,10,TACC,189,1782,4,6,0,0
7,3,6,TOKY,356,1370,0,0,4,0
8,5,10,HAWI,504,1846,0,0,0,0
9,3,6,MASS,366,1394,2,3,0,0


## Step-2: Creating a cluster from an instance type.

In [40]:
# Run this cell
# Initialize the variables appropriately. 
# Number of nodes in the cluster.
num_nodes=6

# Give a cluster name
slice_name='DeepPrep'

# Make it True if you want to include persistence storage to a single node. KEEP IT FALSE FOR THE TIME BEING.
storage=False

# get attached to the cluster. By default adding NVMe is false.
add_NVMe = True

# Make it True if you want to include GPUs to different nodes in the cluster [depends on availability].
add_gpu=True

# By default master node will not have GPUs in it. For CPU only cluster False means lower resources then the workers.
master_gpu=False

# If only t4 gpu needs to be added.
add_t4=True

# If only rtx gpu needs to be added.
add_rtx=True

# If only rtx gpu needs to be added.
add_a40=False


# Site name, pick one site from the above list of resources.
site='GPN'

# Select node resource:- It follows the pattern fabric.c#N.m#N.d#N , where c:cores, m:primary memory, d:disk and #N: capacity.
# eg: fabric.c4.m8.d50 means cores:4, Memory:8Gb, disk:50Gb  
# List of node instance type is provided in - https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json
#instance_worker='fabric.c24.m128.d500'
instance_worker='fabric.c24.m64.d1000'

# The resources of master node. It is better to be less than the worker nodes.
instance_master='fabric.c8.m64.d500'

# Operation system, Linux distribution e.g. default_ubuntu_18, default_ubuntu_20, etc.
image='default_ubuntu_20'


In [41]:
import pandas as pd

try:
    json_format=fablib.list_sites(output='json',quiet=True)
except Exception as e:
    print(f"Exception: {e}")

sites_df=pd.read_json(json_format)
site_df=sites_df[sites_df['name']==site]
type_t4=site_df['tesla_t4_available'].values
type_rtx=site_df['rtx6000_available'].values
type_a40=site_df['a40_available'].values

nvme_available=site_df['nvme_available'].values
print("Number of Nvidia t4 available now at",site,"is:", type_t4[0])
print("Number of Nvidia rtx6000 available now at",site,"is:", type_rtx[0])
print("Number of Nvidia A40 available now at",site,"is:", type_a40[0])
print("Number of NVMe available now at",site,"is:", nvme_available[0])

max_type_t4 = type_t4[0]
max_type_rtx = type_rtx[0]
max_type_a40 = type_a40[0]

total_gpus = max_type_t4 + max_type_rtx + max_type_a40



Number of Nvidia t4 available now at GPN is: 3
Number of Nvidia rtx6000 available now at GPN is: 6
Number of Nvidia A40 available now at GPN is: 0
Number of NVMe available now at GPN is: 16


/tmp/ipykernel_79/299778379.py:8: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  sites_df=pd.read_json(json_format)


In [42]:
# The max available GPU for the chosen site is mentioned above, to change it please make changes to the variables below.
# By default initialized with the available GPUs for the site.

custom_number_of_t4 = max_type_t4
custom_number_of_rtx = max_type_rtx
custom_number_of_a40 = max_type_a40


# custom_number_of_t4 = 3
# custom_number_of_rtx =4

if custom_number_of_t4 < max_type_t4 or custom_number_of_rtx < max_type_rtx:
    max_type_t4 = custom_number_of_t4
    max_type_rtx = custom_number_of_rtx
    total_gpus = max_type_t4 + max_type_rtx
print(total_gpus)
gpu_names=[]

for i in range(1,total_gpus+1):
    gpu_names.append("GPU_{0}".format(i))
 
print(gpu_names)
temp_t4=max_type_t4
temp_rtx=max_type_rtx

######################################################
#####                  NVMe                      #####
######################################################

# Assumption: If the number of available NVMes' are less than the number of cluster node minus the master NVMes' will not

if nvme_available >= num_nodes-1 and add_NVMe == True:
    add_NVMe = True
print(add_NVMe)


9
['GPU_1', 'GPU_2', 'GPU_3', 'GPU_4', 'GPU_5', 'GPU_6', 'GPU_7', 'GPU_8', 'GPU_9']
True


In [43]:
# Run this cell

node_names=[]
nic_names=[]
iface_names=[]
nvme_names=[]
network_name='cluster_network'
storage_name = 'gaf-storage'

for i in range(1,num_nodes+1):
    node_names.append("Node{0}".format(i))
    nic_names.append("Nic{0}".format(i))
    iface_names.append("iface{0}".format(i))
    nvme_names.append("nvme{0}".format(i))

print(node_names)
print(nic_names)
print(iface_names)
print(nvme_names)

['Node1', 'Node2', 'Node3', 'Node4', 'Node5', 'Node6']
['Nic1', 'Nic2', 'Nic3', 'Nic4', 'Nic5', 'Nic6']
['iface1', 'iface2', 'iface3', 'iface4', 'iface5', 'iface6']
['nvme1', 'nvme2', 'nvme3', 'nvme4', 'nvme5', 'nvme6']


In [44]:
# Only run when persistence storage needed to be attached. 
if storage == True:
    import traceback
    from plugins import Plugins
    try:
        Plugins.load()
    except Exception as e:
        traceback.print_exc()
print(temp_t4)
print(temp_rtx)


3
6


In [45]:
print(add_a40)

False


In [46]:
# # Run this cell, Visit the link below to find different instance type options. 
# # Read the comments carefully given below and make changes as necessary. 
# ## Allocating 2 A40 GPUs on VM1 (worker node) and VM0 as managing node
# try:
#     slice=fablib.new_slice(name=slice_name)
#     for i in range(num_nodes):
#         if master_gpu == False and i == 0:
#             node=slice.add_node(name=node_names[i],
#                                 site=site, 
#                                 instance_type=instance_master,
#                                 image=image)
#             iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
#         else:
#             node=slice.add_node(name=node_names[i],
#                                 site=site,
#                                 instance_type=instance_worker,
#                                 image=image)
#             iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]
#             if add_gpu == True:
#                 print(i)
#                 if add_a40 == True  and i == 1: # Attach 2-A40 GPUs on VM1 machine
#                     node.add_component(model='GPU_A40', name=gpu_names[i-1])
#                     node.add_component(model='GPU_A40', name=gpu_names[i-2])
#                     print("allocated 2 A40 GPUs")
#                     type_a40 = type_a40 - 2 
#                 else: 
#                     print("No GPUs available")
#             if add_NVMe == True:
#                 node.add_component(model='NVME_P4510', name=nvme_names[i-1])
#             else:
#                 print("NVMe not avail")
# except Exception as e:
#     print(f'Exception: {e}') 

In [47]:
# # Run this cell, Visit the link below to find different instance type options. 
# # Read the comments carefully given below and make changes as necessary. 

# try:
#     slice=fablib.new_slice(name=slice_name)
    
#     for i in range(num_nodes):
#         if master_gpu == False and i == 0:
#             node=slice.add_node(name=node_names[i],
#                                 site=site, 
#                                 instance_type=instance_master,
#                                 image=image)
#             iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
        
#         else:
#             node=slice.add_node(name=node_names[i],
#                                 site=site,
#                                 instance_type=instance_worker,
#                                 image=image)
#             iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]
            
#             if add_gpu == True:
#                 print(i)
#                 if add_t4 == True and temp_t4 > 1 and i==1: # allocating 2 T4
#                     node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
#                     node.add_component(model='GPU_TeslaT4', name=gpu_names[i-2])
#                     temp_t4 = temp_t4 - 2 
#                     print("allocated 2T4")
#                 if add_t4 == True and temp_t4 > 0 and i==2: # allocating 1 T4
#                     node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
#                     temp_t4 = temp_t4 - 1 
#                     print("allocated 1T4")
#                 # if add_rtx == True and temp_rtx >2 and i==3: # allocating 3rtx
#                 #     node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
#                 #     node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
#                 #     node.add_component(model='GPU_RTX6000',name=gpu_names[i-3])
#                 #     temp_rtx = temp_rtx - 3
#                 #     print("allocated 3rtx")
#                 if add_rtx == True and temp_rtx >1 and i==3: # allocating 2 T4
#                     node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
#                     node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
#                     temp_rtx = temp_rtx - 2
#                     print("allocated 2rtx")
#                 if add_rtx == True and temp_rtx >0 and i==4: # allocating 1 rtx
#                     node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
#                     temp_rtx = temp_rtx - 1
#                     print("allocated 1rtx")
#                 else: 
#                     print("No GPUs available")
#             if add_NVMe == True:
#                 node.add_component(model='NVME_P4510', name=nvme_names[i-1])
# except Exception as e:
#     print(f'Exception: {e}') 

In [48]:
# Run this cell, Visit the link below to find different instance type options. 
# Read the comments carefully given below and make changes as necessary. 

try:
    slice=fablib.new_slice(name=slice_name)
    
    for i in range(num_nodes):
        if master_gpu == False and i == 0:
            node=slice.add_node(name=node_names[i],
                                site=site, 
                                instance_type=instance_master,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic', name=node_names[i]).get_interfaces()[0]
        
        else:
            node=slice.add_node(name=node_names[i],
                                site=site,
                                instance_type=instance_worker,
                                image=image)
            iface_names[i]=node.add_component(model='NIC_Basic',name=node_names[i]).get_interfaces()[0]
            
            if add_gpu == True:
                print(i)
                if add_t4 == True and temp_t4 > 1 and i==1: # allocating 2 T4
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-2])
                    temp_t4 = temp_t4 - 2 
                    print("allocated 2T4")
                if add_t4 == True and temp_t4 > 0 and i==2: # allocating 1 T4
                    node.add_component(model='GPU_TeslaT4', name=gpu_names[i-1])
                    temp_t4 = temp_t4 - 1 
                    print("allocated 1T4")
                if add_rtx == True and temp_rtx >2 and i==3: # allocating 3rtx
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-3])
                    temp_rtx = temp_rtx - 3
                    print("allocated 3rtx")
                if add_rtx == True and temp_rtx >1 and i==4: # allocating 2 T4
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-2])
                    temp_rtx = temp_rtx - 2
                    print("allocated 2rtx")
                if add_rtx == True and temp_rtx >0 and i==5: # allocating 1 rtx
                    node.add_component(model='GPU_RTX6000',name=gpu_names[i-1])
                    temp_rtx = temp_rtx - 1
                    print("allocated 1rtx")
                else: 
                    print("No GPUs available")
            if add_NVMe == True:
                node.add_component(model='NVME_P4510', name=nvme_names[i-1])
            else:
                print("NVMe not avail")
except Exception as e:
    print(f'Exception: {e}') 


1
allocated 2T4
No GPUs available
2
allocated 1T4
No GPUs available
3
allocated 3rtx
No GPUs available
4
allocated 2rtx
No GPUs available
5
allocated 1rtx


In [49]:
# Run this cell
try:
    net_cluster=slice.add_l2network(name=network_name, interfaces=iface_names[:])
except Exception as e:
    print(f"Exception: {e}")

In [50]:
# Run this cell
# If this cell get executed successfully then IP address of the nodes will be displayed which can be used to ssh into the respective nodes.
# If there is an error while creating the slice/cluster, repeat from the third code cell block.
try:
    slice.submit()
except Exception as e:
    print(f'Exception: {e}')
    


Retry: 16, Time: 413 sec


ID,6bc62d55-4cdb-49c0-a07c-c8391e2b630a
Name,DeepPrep
Lease Expiration (UTC),2025-10-01 21:35:28 +0000
Lease Start (UTC),2025-09-30 21:35:28 +0000
Project ID,68926660-da26-475d-9c40-50ebf0a5a812
State,StableOK
Email,ajay.kumar@missouri.edu
UserId,2b184bdd-e606-4190-9ff5-f6ec655a6dfc


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
83bc66ff-e55d-4e50-bd1f-b414e7c78063,Node1,8,64,500,default_ubuntu_20,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe97:a415,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe97:a415,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
e1292a49-ca76-494a-afab-ac0fc396f944,Node2,24,64,1000,default_ubuntu_20,qcow2,gpn-w5.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe23:2d5c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe23:2d5c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
b70c2ea7-985d-4105-a01b-6058de19c833,Node3,24,64,1000,default_ubuntu_20,qcow2,gpn-w4.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fec8:2ed7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fec8:2ed7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
efe28fe7-ce12-4820-b4b0-878da2b6ba1f,Node4,24,64,1000,default_ubuntu_20,qcow2,gpn-w2.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe9c:f40f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe9c:f40f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
57bfec84-1cfb-45d8-9e6d-916c8ee2566d,Node5,24,64,1000,default_ubuntu_20,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe2b:e5a7,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe2b:e5a7,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
c5836429-c56e-403b-86ef-6df4d4a614c3,Node6,24,64,1000,default_ubuntu_20,qcow2,gpn-w1.fabric-testbed.net,GPN,ubuntu,2610:e0:a04c:fab2:f816:3eff:fe74:e91f,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:e0:a04c:fab2:f816:3eff:fe74:e91f,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
b2cfa52f-a419-4086-9ff3-6e40049fab3c,cluster_network,L2,L2Bridge,GPN,None,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
Node1-Node1-p1,p1,Node1,cluster_network,100,config,,02:0F:7E:EA:E5:ED,enp7s0,enp7s0,fe80::f:7eff:feea:e5ed,6,HundredGigE0/0/0/7
Node2-Node2-p1,p1,Node2,cluster_network,100,config,,0A:3D:4E:3E:91:96,enp10s0,enp10s0,fe80::83d:4eff:fe3e:9196,4,HundredGigE0/0/0/13
Node3-Node3-p1,p1,Node3,cluster_network,100,config,,02:14:79:AE:DA:CC,enp7s0,enp7s0,fe80::14:79ff:feae:dacc,4,HundredGigE0/0/0/11
Node4-Node4-p1,p1,Node4,cluster_network,100,config,,02:49:03:96:49:74,enp11s0,enp11s0,fe80::49:3ff:fe96:4974,6,HundredGigE0/0/0/7
Node5-Node5-p1,p1,Node5,cluster_network,100,config,,02:31:E6:AF:DC:68,enp10s0,enp10s0,fe80::31:e6ff:feaf:dc68,6,HundredGigE0/0/0/5
Node6-Node6-p1,p1,Node6,cluster_network,100,config,,02:19:1B:38:14:52,enp7s0,enp7s0,fe80::19:1bff:fe38:1452,6,HundredGigE0/0/0/5



Time to print interfaces 423 seconds


## Step-3: Configuring the network and setting up the cluster

In [51]:
# try:
#     slice=fablib.get_slice(name="GPU_Variant_Calling_FIU") # Put the cluster name that you want to delete
#     # slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")

In [52]:
#Run this cell
from ipaddress import IPv4Address, IPv4Network
try:
    subnet=IPv4Network("192.168.1.0/24")
    available_ips=list(subnet)[1:]
except Exception as e:
    print(f"Exception: {e}")

In [53]:
try:
   for node in slice.get_nodes():
       node_iface=node.get_interface(network_name=network_name)
       # print(node_iface)
       stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
       # print(stdout,stderr)
except Exception as e:
   print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:0f:7e:ea:e5:ed brd ff:ff:ff:ff:ff:ff
    inet6 fe80::f:7eff:feea:e5ed/64 scope link 
       valid_lft forever preferred_lft forever
3: enp10s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0a:3d:4e:3e:91:96 brd ff:ff:ff:ff:ff:ff
    inet6 fe80::83d:4eff:fe3e:9196/64 scope link 
       valid_lft forever preferred_lft forever
3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:14:79:ae:da:cc brd ff:ff:ff:ff:ff:ff
    inet6 fe80::14:79ff:feae:dacc/64 scope link 
       valid_lft forever preferred_lft forever
3: enp11s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:49:03:96:49:74 brd ff:ff:ff:ff:ff:ff
    inet6 fe80::49:3ff:fe96:4974/64 scope link 
       valid_lft forever preferred_lft forever
3: enp10s0:

In [54]:
#Run this cell
#%%capture
try:
    for node in slice.get_nodes():
        node_iface=node.get_interface(network_name=network_name)
        node_IP_addr=available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_IP_addr, subnet=subnet)
        stdout, stderr = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        _, _ = node.execute('sudo apt-get update')
        stdout, stderr = node.execute('sudo apt install net-tools')        
        stdout, stderr = node.execute(f'sudo ifconfig {node_iface.get_os_interface()} up')
except Exception as e:
    print(f"Exception: {e}")

3: enp7s0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 02:0f:7e:ea:e5:ed brd ff:ff:ff:ff:ff:ff
    inet 192.168.1.1/24 scope global enp7s0
       valid_lft forever preferred_lft forever
    inet6 fe80::f:7eff:feea:e5ed/64 scope link 
       valid_lft forever preferred_lft forever
Hit:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:8 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 

In [55]:
#Run this cell
# If the ping is successful, that means the nodes are connected properly if there is any error then may need to recreate the cluster or further have to look into it. 
try:
    node1=slice.get_node(name='Node1')
    stdout,stderr=node1.execute(f' ping -c 3 192.168.1.4')
    print(stdout)
    print(stderr)
except Exception as e:
    print(f'Exception: {e}')


PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
64 bytes from 192.168.1.4: icmp_seq=1 ttl=64 time=0.331 ms
64 bytes from 192.168.1.4: icmp_seq=2 ttl=64 time=0.123 ms
64 bytes from 192.168.1.4: icmp_seq=3 ttl=64 time=0.096 ms

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2040ms
rtt min/avg/max/mdev = 0.096/0.183/0.331/0.105 ms
PING 192.168.1.4 (192.168.1.4) 56(84) bytes of data.
64 bytes from 192.168.1.4: icmp_seq=1 ttl=64 time=0.331 ms
64 bytes from 192.168.1.4: icmp_seq=2 ttl=64 time=0.123 ms
64 bytes from 192.168.1.4: icmp_seq=3 ttl=64 time=0.096 ms

--- 192.168.1.4 ping statistics ---
3 packets transmitted, 3 received, 0% packet loss, time 2040ms
rtt min/avg/max/mdev = 0.096/0.183/0.331/0.105 ms




In [56]:
# Run this cell
# Function to create a file that contains IPs and hostnames related to it.

def append_line(file_path,text):
    with open(file_path,"a+") as file_des:
        file_des.seek(0)
        data=file_des.read(-1)
        if len(data)>0:
            file_des.write("\n")
        file_des.write(text)

In [57]:
# Run this cell

import os

if os.path.isfile('/home/fabric/work/hosts') or os.path.isfile('/home/fabric/work/ips.txt') or os.path.isfile('/home/fabric/work/workers'):
    os.system("rm /home/fabric/work/hosts")
    os.system("rm /home/fabric/work/ips.txt")
    os.system("rm /home/fabric/work/workers")    
else:
    print("does not exist")

if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
    os.system("rm /home/fabric/work/gpu_ips.txt")
    
#os.system("rm /home/fabric/work/ips.txt")


rm: cannot remove '/home/fabric/work/ips.txt': No such file or directory
rm: cannot remove '/home/fabric/work/workers': No such file or directory


In [ ]:
# Run this cell

# Capturing the IP addresses, this step needs to be integrated with the IP assigning stage, coded above. Or it may stay independent.
import os

i=1
local_host="127.0.0.1 localhost"
path_to_host_file="/home/fabric/work/hosts"
path_to_ip_file="/home/fabric/work/ips.txt"
path_to_worker_ip="/home/fabric/work/workers"
path_to_gpu_ips="/home/fabric/work/gpu_ips.txt"
gpu_name="NVIDIA"

append_line(path_to_host_file,local_host)

try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute("hostname -I")
        IP=stdout.split(" ")[1]
        node_name="node{0}".format(i)
        vm_names="vm{0}".format(i-1)
        append_line(path_to_ip_file,IP)
        
        stdout, stderr=node.execute("hostname")
        line=IP+" "+node_name+" "+vm_names+" "+stdout
        append_line(path_to_host_file,line)
      
    
        if(i>1):
            append_line(path_to_worker_ip,vm_names)
            #append_line(path_to_worker_ip,IP)
        
        print(line)
        print(stderr)
        
        stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
        if gpu_name in stdout_gpu:
            stdout_ip, _=node.execute('hostname -I')
            ip=stdout_ip.split(" ")[1]
            append_line(path_to_gpu_ips,ip)
            #print(ip)
        
        i=i+1
except Exception as e:
    print(f"Exception: {e}")
    
print(IP)

10.30.6.215 192.168.1.1 2610:e0:a04c:fab2:f816:3eff:fe97:a415 
Node1
192.168.1.1 node1 vm0 Node1


10.30.6.12 192.168.1.2 2610:e0:a04c:fab2:f816:3eff:fe23:2d5c 
Node2
192.168.1.2 node2 vm1 Node2


07:00.0 3D controller: NVIDIA Corporation TU104GL [Tesla T4] (rev a1)
08:00.0 3D controller: NVIDIA Corporation TU104GL [Tesla T4] (rev a1)
10.30.6.12 192.168.1.2 2610:e0:a04c:fab2:f816:3eff:fe23:2d5c 
10.30.6.214 192.168.1.3 2610:e0:a04c:fab2:f816:3eff:fec8:2ed7 


In [ ]:
# Run this cell
try:
    for node in slice.get_nodes():
        stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') # if you run the command twice the back up will be overwritten, a conditional block should be written
        output_host_copy=node.upload_file(path_to_host_file,"/home/ubuntu/hosts")
        output_ip_copy=node.upload_file(path_to_ip_file,"/home/ubuntu/ips.txt")
        output_worker_copy=node.upload_file(path_to_worker_ip,"/home/ubuntu/workers")
        stdout_host_copy,stderr_host_copy=node.execute(f'sudo cp /home/ubuntu/hosts /etc/hosts')
        if os.path.isfile('/home/fabric/work/gpu_ips.txt'):
            output_gpu_copy=node.upload_file(path_to_gpu_ips,"/home/ubuntu/gpu_ips.txt")
        print(stderr)
        print(stderr_host_copy)
except Exception as e:
    print(f"Exception : {e}")
    

In [ ]:
# Run this cell

import os

output=os.system('ssh-keygen -q -t rsa -N "" -f /home/fabric/work/id_rsa > /dev/null 2>&1')

In [ ]:
# Run this cell

try:
    for node in slice.get_nodes():
        output_private=node.upload_file("/home/fabric/work/id_rsa","/home/ubuntu/.ssh/id_rsa")
        output_public=node.upload_file("/home/fabric/work/id_rsa.pub","/home/ubuntu/.ssh/id_rsa.pub")
        
        stdout, stderr=node.execute(f' cat /home/ubuntu/.ssh/id_rsa.pub >> /home/ubuntu/.ssh/authorized_keys')
        stdout, stderr=node.execute(f' chmod 600 /home/ubuntu/.ssh/id_rsa*')
        
        print(output_private)
        print(output_public)
        print(stderr)
        #print(stdout)
        
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
# Run this cell. This is the last cell to run. Please read the comments in the next few cells to know how to extend the lease time and how to delete a slice/cluster. 

from ipaddress import ip_address, IPv6Address
try:
    for node in slice.get_nodes():
        if type(ip_address(node.get_management_ip())) is IPv6Address:
            node.upload_file("/home/fabric/work/nat64.sh", "/home/ubuntu/nat64.sh")
            #stdout, stderr=node.execute(f' chmod +x /home/ubuntu/nat64.sh && sudo bash /home/ubuntu/nat64.sh')
            stdout, stderr=node.execute(f' chmod +x /home/ubuntu/nat64.sh && sudo bash /home/ubuntu/nat64.sh > /dev/null 2>&1')
            print(stdout)
            print(stderr)
except Exception as e:
    print(f"Exception: {e}")
    

## Step-4: Deleting and extending lease of a slice

In [ ]:
# # Run this cell ONLY when you want to delete the cell
# # To delete a slice/cluster.
# try:
#     slice=fablib.get_slice(name="AVAH") # Put the cluster name that you want to delete
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")

In [ ]:
# # Run this cell ONLY when you want to exted the time of the slice. Or to extend the lease time of the slice/cluster.

# import datetime
# slice_name='demo' # Give the cluster/slice name that you want to extend

# #Set end host to now plus 1 day
# new_end_date = (datetime.datetime.utcnow() + datetime.timedelta(days=7)).strftime("%Y-%m-%d %H:%M:%S %z")
# #new_end_date = (datetime.now(timezone.utc) + timedelta(days=6)).strftime("%Y-%m-%d %H:%M:%S %z")
# print(type(new_end_date), new_end_date)
# try:
#     slice=fablib.get_slice(name=slice_name)
#     slice.renew('2023-05-31 18:04:26 +0000') # Give the new lease end date and time of the slice. One can increase it by 7 days from the day of creation of the slice/cluster. 
    
#     #print(f"Lease End (UTC)        : {slice.get_lease_end()}")
# except Exception as e:
#     print(f"Exception: {e}")

In [ ]:
# # Run this cell ONLY to observe the new lease time.

# slice_name='cluster_exp' # Give the slice/cluster name that you just extended. 
# try:
#     slice = fablib.get_slice(name=slice_name)
#     print(f"{slice}")
# except Exception as e:
#     print(f"Exception: {e}")

In [ ]:
# # Run this cell ONLY to observe the new lease time.

# slice_name='cluster_gpu' # Give the slice/cluster name that you just extended. 
# try:
#     slice = fablib.get_slice(name=slice_name)
#     gpu_name="NVIDIA"
#     for node in slice.get_nodes():
#         #stdout, stderr=node.execute(f'sudo cp /etc/hosts /etc/hosts_backup') # if you run the command twice the back up will be overwritten, a conditional block should be written
#         stdout_gpu, _=node.execute('lspci | grep NVIDIA')   
#         if gpu_name in stdout_gpu:
#             stdout_ip, _=node.execute('hostname -I')
#             ip=stdout_ip.split(" ")[1]
#             print(ip)
# except Exception as e:
#     print(f"Exception: {e}")